In [ ]:
# For Google Colab to access Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# David Cabezas Berrido

# p2pGAN_test-results.py

# Generates predictions for test images and writes results to disc

import tensorflow as tf
import matplotlib.pyplot as plt
import os

PATH = '.' #'/content/drive/My Drive/TFG-Image-Optimization' # for Google Colab
INPATH = PATH+'/data/jpeg/short/'
MODELS = PATH+'/models/'
OUPATH = PATH+'/results/p2pGAN/'

#OUPATH = PATH+'/results/p2pGANlow/' # Processed by generator with lower resolution (1024 x 1536)
#OUPATH = PATH+'/results/p2pGANlow2/' # Processed by generator with lower resolution (2048 x 3072)
#OUPATH = PATH+'/results/p2pGANepoch70/'
#OUPATH = PATH+'/results/p2pGANepoch60/'
#OUPATH = PATH+'/results/p2pGAN-train-false/'


imgurls = !ls -1 "{INPATH}" # short exposure images

# Match each short exposure image with the corresponding long exposure image 
def urlTarget(url):
    ext=url.split('.')[-1]
    u=url.split('-')
    return ''.join([u[0],'-',u[1],'.'+ext])

imgurls_tg=[(url,urlTarget(url)) for url in imgurls]

# Train/Test split
def isForTrain(url):
    u=url.split('-')
    return u[1][0] in {'0','1'}

# Test images
ts_urls = [url for url in imgurls_tg if not isForTrain(url[0])] # First digit is 2

# Dimensions for test images
#TEST_HEIGHT = 3072 #=12*256 # ! This resolution requires a large RAM size
#TEST_WIDTH = 4608 #=18*256

#TEST_HEIGHT = 2048 
#TEST_WIDTH = 3072

#TEST_HEIGHT = 1024 
#TEST_WIDTH = 1536

In [ ]:
# Load generator
generator = tf.keras.models.load_model(MODELS+'GAN-generator')
#generator = tf.keras.models.load_model(MODELS+'GAN-generator-epoch60')
#generator = tf.keras.models.load_model(MODELS+'GAN-generator-epoch70')

In [ ]:
# Function to read test images
def decode_normalize(path):
    inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(path)),tf.float32)[...,:3]
    inimg = tf.image.resize(inimg, [TEST_HEIGHT, TEST_WIDTH], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    inimg = inimg/127.5-1
    return inimg

In [ ]:
for inimg, target in ts_urls: # Iterate throgh input test images
    dat = tf.data.Dataset.from_tensor_slices([INPATH+inimg])
    dat = dat.map(decode_normalize)
    dat = dat.batch(1)
    for img in dat:
        prediction = generator(img, training=True) # Prediction for input
        # getting the pixel values between [0, 1]
        #plt.imshow(prediction[0]*0.5+0.5) # Plot prediction
        #plt.show()
        tf.keras.preprocessing.image.save_img(OUPATH+inimg,prediction[0]*0.5+0.5) # Save to disc